In [1]:
import pandas as pd
import numpy as np
import re

In [32]:
nywf=pd.read_csv('nywf.csv')
census=pd.read_csv('census.csv')

In [10]:
#question 1
#Count different providers
providers=pd.unique(nywf['Provider'])
len(providers)

17

In [15]:
#question 2
# Count providers and amount of hotspots in the Bronx.
Bronxwf=nywf[nywf['Borough']==2]
BronxGroups=Bronxwf.groupby('Provider')
for x,y in BronxGroups:
    print(x,len(y))

ALTICEUSA 120
AT&T 3
LinkNYC - Citybridge 137
NYPL 35
Transit Wireless 21


In [46]:
#question 3
#Define a function to determine if the hotspot is in a park.
def InPark(row):
    if re.search('[Pp][Aa][Rr][Kk][Ss]*(?![\w])',str(row['Location'])):
        return True
    return False
#create a new column with boolean value.
nywf['Park']=nywf.apply(InPark,axis=1)
#Calculate the fraction
print(len(nywf[nywf['Park']])/len(nywf))

0.04579692678517626


In [21]:
#question 4
#Define a function to determine if the hotspot is in a library.
def InLibrary(row):
    if re.search('[Ll][Ii][Bb][rR][aA][rR][yY]',str(row['Location_T'])):
        return True
    return False
#create a new column with boolean value.
nywf['Library']=nywf.apply(InLibrary,axis=1)
#select the hotspots that are not in libraries
nonlib=nywf[nywf['Library']==False]
#Print the proportion of unlimited free hotspots
print(len(nonlib[nonlib['Type']=="Free"])/len(nonlib))

0.8121778350515464


In [45]:
#question 5
#rename the NTACODE column of the NY wifi dataframe:
columns=list(nywf.columns)
columns[18]='NTACODE'
nywf.columns=columns
#group by NTACODE
NTACODEGroups=nywf.groupby('NTACODE')
#select the ones with more than 30 hotspots (HS)
LotOfHS=[(x,len(y)) for x,y in NTACODEGroups if len(y)>=30]
lotsofHS=pd.DataFrame(LotOfHS,columns=['NTACODE','HSamount'])
#rename some columns of the NY population dataframe:
columns=list(census.columns)
columns[2]='NTACODE'
columns[5]='TP2010'
census.columns=columns
#intersect the dataframes on the NTACODE column:
CensusPlus=pd.merge(census,lotsofHS,how='inner',on='NTACODE')
#define a function to compute HS per capita and add a column with the info
def HSperCap(row):
    return row['HSamount']/row['TP2010']
CensusPlus['HSperCap']=CensusPlus.apply(HSperCap,axis=1)
#get some statistics
CensusStats=CensusPlus.describe()
#print the interquartile range
print(CensusStats.loc['75%','HSperCap']-CensusStats.loc['25%','HSperCap'])

0.0008508586597922147


In [60]:
#question 6
#Convert the coordinates to radians
def Radians(row):
    radlat=float(row['Latitude'])*np.pi/180
    radlon=float(row['Longitude'])*np.pi/180
    return (radlat,radlon)
nywf['Radians']=nywf.apply(Radians,axis=1)
#define a function to compute the distance
R=6371*3280.84
def distance(p1,p2):
    latm=(p1[0]+p2[0])/2
    return R*np.sqrt((p2[0]-p1[0])**2+(np.cos(latm)*(p1[1]-p2[1]))**2)
#We will take a sample to compute the median of the distances
CoordsDF=pd.merge(CensusPlus,nywf,how='inner',on='NTACODE')
CoordSample=CoordsDF.sample(n=500)
def Vecinos(row):
    vecinos=set([50000,50001,50002])
    code=row['NTACODE']
    samecodeDF=CoordsDF[CoordsDF['NTACODE']==code]
    mycoords=row['Radians']
    for i in samecodeDF.index:
        dist=distance(mycoords,samecodeDF.loc[i]['Radians'])
        if dist<max(vecinos) and dist!=0:
            vecinos.discard(max(vecinos))
            vecinos.add(dist)
    return sum(vecinos)/len(vecinos)

CoordSample['Average Distance']=CoordSample.apply(Vecinos,axis=1)

In [63]:
CoordSample.describe().loc['50%','Average Distance']

272.0109367549215